In [1]:
import numpy as np
from pcm_fmm import pcm_fmm
pcm_fmm.init_globals()

In [2]:
lmax = 100
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1.5
sph2_c = np.array([2.0, 1, 2])
sph2_r = sph1_r + np.linalg.norm(sph2_c)
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([4.4, 5.5, 6.6])
print(sph2_r, np.linalg.norm(dst1-sph2_c))
coef1_m = pcm_fmm.fmm_p2m(src1-sph1_c, sph1_r, lmax)
coef2_m = pcm_fmm.fmm_p2m(src1-sph2_c, sph2_r, lmax)
print(pcm_fmm.fmm_m2p(dst1-sph1_c, sph1_r, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(pcm_fmm.fmm_m2p(dst1-sph2_c, sph2_r, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef3_m = np.zeros_like(coef2_m)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, coef1_m, coef3_m)
print(np.linalg.norm(coef2_m-coef3_m)/np.linalg.norm(coef2_m))

4.5 6.868041933477111
0.10845884916465832 0.10845884916465832
0.10845884916465826 0.10845884916465832
4.680830157874228e-16


In [10]:
N = 20
lmax = 10
csph = np.random.randn(3, N)
rsph = np.ones(N)
ind = np.arange(1, N+1, dtype=np.int32)
cluster, children, parent, cnode, rnode = pcm_fmm.tree_init(csph, rsph, ind)
#print(cluster, children, parent, cnode, rnode)
coef_sph = np.random.randn((lmax+1)*(lmax+1), N)
coef_node = pcm_fmm.tree_m2m_baseline(lmax, coef_sph, ind, cluster, children, cnode, rnode)
dst = np.random.rand(3)
dst = dst / np.linalg.norm(dst) * (1+np.max(np.linalg.norm(csph, axis=0))) * 2
v0 = pcm_fmm.fmm_m2p(dst-cnode[:,0], rnode[0], lmax, coef_node[:,0])
v1 = 0
for i in range(N):
    v1 += pcm_fmm